In [68]:
# from preprocessing.data_collection.multipleye_data_collection import prepare_language_folder
from preprocessing.data_collection.multipleye_data_collection import MultipleyeDataCollection
from pathlib import Path

## Pre-processing MultiplEYE Data

In [69]:
data_collection_name = 'MultiplEYE_SQ_CH_Zurich_1_2025'
# data_collection_name = 'MultiplEYE_SL_SI_Ljubljana_1_2025'

If necessary, prepare the data folder by unzipping the downloaded files. Works only for MultiplEYE and MeRID data collections so far. Also, there might be some manual steps necessary.

In [70]:
this_repo = Path().resolve()
data_folder_path = this_repo / "data" / data_collection_name

# MultipleyeDataCollection.create_from_data_folder(data_folder_path)


In [ ]:
multipleye_sq = MultipleyeDataCollection.create_from_data_folder(data_folder_path)

preprocessed_data_folder = this_repo / "preprocessed_data" / data_collection_name
preprocessed_data_folder.mkdir(parents=True, exist_ok=True)

In [72]:
multipleye_sq

Title	MultiplEYE_SQ_CH_Zurich_1_2025
Dataset_type	MultiplEYE
Number_of_sessions	9
Number_of_pilots	0
Tested_language	SQ
Country	CH
Year	2025
Number of eye-tracking (ET) sessions per participant	1

In [73]:
sessions = [s for s in multipleye_sq]
sess = sessions[0]
idf = sess.session_identifier

## Creating Gaze Frame from ASCII File

In [74]:
from preprocessing import peyepeline
from preprocessing import config

asc = sess.asc_path
output_folder = preprocessed_data_folder / idf
output_folder.mkdir(parents=True, exist_ok=True)

In [75]:
gaze = peyepeline.load_gaze_data(
    asc_file=asc,
    lab_config=sess.lab_config,
    session_idf=idf,
    trial_cols=config.TRIAL_COLS,
)

In [ ]:
peyepeline.save_raw_data(output_folder / 'raw_data', sess.session_identifier, gaze)

## Coordinate and Velocity Preprocessing

Eye movements are recorded in screen pixel coordinates, which depend on stimulus size and monitor setup. To compare gaze behavior across participants, screens, or datasets, it is standard to convert pixel positions 
into **degrees of visual angle (dva)**. Next, we compute **gaze velocity**, which allows us to detect saccades and distinguish them from fixations.

In [77]:
peyepeline.preprocess_gaze(gaze)

## Detect Events and Compute Their Properties

Eye-tracking data are typically segmented into events, i.e. `fixations` and `saccades`. Fixations represent moments when the eyes remain relatively still, allowing visual information to be processed, while saccades are the rapid movements between fixations that reposition the gaze. Detecting these events and computing their properties, such as `dispersion`, fixation `duration`, saccade `amplitude`, and `peak velocity`, provides the foundation for analyzing visual behavior and understanding how participants explore a stimulus.

### Fixations

We can detect fixations by applying the `I-VT` or the `I-DT` method.

The **I-VT (Velocity-Threshold Identification)** method distinguishes fixation and saccade points based on their point-to-point velocities. Each point is classified as a fixation if its velocity is below the specified threshold. Consecutive fixation points are then merged into a single fixation. A threshold of 20 degrees/second is commonly used as a default maximum value. Read more about [the IVT algorithm in the documentation](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.ivt.html) 

The **I-DT (Dispersion-Threshold Identification)** method finds fixations by grouping consecutive points within a maximum separation (dispersion) threshold and a minimum duration threshold. The algorithm slides a moving window across the data: if the dispersion within the window is below the threshold, the window represents a fixation and is gradually expanded until the dispersion exceeds the threshold.
Read more about [our implementation of the IDT method](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.idt.html).

We use the `I-VT` algorithm with the following key deafault parameters:
- `minimum duration`: 100 ms 
- `velocity threshold`: 20.0

Such properties as `location`, containing the centroid coordinates of each fixation, and `dispersion` will also be calculated.

In [78]:
peyepeline.detect_fixations(
    gaze,
)

### Saccades

Saccades are rapid eye movements that shift the point of fixation from one location to another. We detect saccades (or micro-saccades) from the velocity sequence of gaze data using the [microsaccades algorithm](https://pymovements.readthedocs.io/en/stable/reference/api/pymovements.events.detection.microsaccades.html#pymovements.events.detection.microsaccades). This algorithm implements a noise-adaptive velocity threshold, meaning that the detection threshold automatically scales with the noise level of the velocity signal. Such properties as `amplitude` and `peak velocity` of the detected saccades will also be calcuated.

The key default parameters are:
- `threshold_factor`: Multiplier used to determine the velocity threshold relative to the noise level of the signal. The default value is 6. A higher factor makes the algorithm more conservative (detects fewer saccades), while a lower factor makes it more sensitive.
- `minimum_duration`: Defines how long a velocity peak must persist to be classified as a saccade. The duration is expressed in the same units as timesteps. If no timesteps are provided, the value refers to the number of samples (default = 6), which corresponds to about 12 ms at a 500 Hz sampling rate. Shorter events are ignored as noise. 

In [79]:
peyepeline.detect_saccades(
    gaze,
)

In [80]:
peyepeline.map_fixations_to_aois(
    gaze,
    sess.stimuli,
)

In [81]:
gaze.save(output_folder / 'preprocessed_gaze', save_events=True, save_samples=True, verbose=2)

Saving events to  /Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/preprocessed_data/MultiplEYE_SQ_CH_Zurich_1_2025/006_SQ_CH_1_ET1/preprocessed_gaze/events.feather
Saving samples to /Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/preprocessed_data/MultiplEYE_SQ_CH_Zurich_1_2025/006_SQ_CH_1_ET1/preprocessed_gaze/samples.feather
Saving experiment file to /Users/anastassiashaitarova/Documents/postdoc-life/openEye/multipleye-preprocessing/preprocessed_data/MultiplEYE_SQ_CH_Zurich_1_2025/006_SQ_CH_1_ET1/preprocessed_gaze


time,pupil,stimulus,activity,trial,page,practice,session,pixel,position,velocity
i64,f64,str,str,str,str,bool,str,list[f64],list[f64],list[f64]
2629468,1085.0,"""Enc_WikiMoon_13""","""reading""","""PRACTICE_trial_1""","""page_1""",true,"""006_SQ_CH_1_ET1""","[46.3, 126.2]","[-15.974782, -9.88542]","[-1.204677, -0.592322]"
2629469,1093.0,"""Enc_WikiMoon_13""","""reading""","""PRACTICE_trial_1""","""page_1""",true,"""006_SQ_CH_1_ET1""","[44.7, 124.9]","[-16.01472, -9.919118]","[-1.191805, -0.608509]"
2629470,1083.0,"""Enc_WikiMoon_13""","""reading""","""PRACTICE_trial_1""","""page_1""",true,"""006_SQ_CH_1_ET1""","[44.1, 126.3]","[-16.029693, -9.882828]","[-1.215066, -0.604523]"
2629471,1088.0,"""Enc_WikiMoon_13""","""reading""","""PRACTICE_trial_1""","""page_1""",true,"""006_SQ_CH_1_ET1""","[44.7, 126.4]","[-16.01472, -9.880235]","[-1.261583, -0.63971]"
2629472,1087.0,"""Enc_WikiMoon_13""","""reading""","""PRACTICE_trial_1""","""page_1""",true,"""006_SQ_CH_1_ET1""","[44.3, 124.7]","[-16.024702, -9.924302]","[-1.251194, -0.60687]"
…,…,…,…,…,…,…,…,…,…,…
10558289,877.0,"""Arg_PISARapaNui_11""","""question""","""trial_10""","""question_11131""",false,"""006_SQ_CH_1_ET1""","[278.9, 887.1]","[-10.015618, 10.229845]","[-0.553149, -0.989099]"
10558290,880.0,"""Arg_PISARapaNui_11""","""question""","""trial_10""","""question_11131""",false,"""006_SQ_CH_1_ET1""","[279.7, 887.1]","[-9.99466, 10.229845]","[-0.434833, -0.882126]"
10558291,878.0,"""Arg_PISARapaNui_11""","""question""","""trial_10""","""question_11131""",false,"""006_SQ_CH_1_ET1""","[278.1, 888.0]","[-10.036573, 10.253125]","[-0.353749, -0.792705]"


## Calculate Reading Measures

In [82]:
from preprocessing.metrics.words import all_words_from_aois, find_skipped_words
from preprocessing.metrics.fixations import annotate_fixations
from preprocessing.metrics.reading_measures import build_word_level_table

import polars as pl

aois = sess.stimuli[4].text_stimulus.aois

### Fixation-based Metrics

In [83]:
fixation_table = annotate_fixations(gaze.events.frame)
fixation_table

fixation_id,trial,stimulus,page,name,onset,offset,duration,dispersion,amplitude,peak_velocity,dispersion_right,location_x,location_y,char_idx,char,top_left_x,top_left_y,width,height,char_idx_in_line,line_idx,word_idx,word_idx_in_line,word,new_visit,visit_id,pass_n
u32,str,str,str,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,i64,str,f64,f64,i64,i64,i64,i64,i64,i64,str,bool,i64,i64
0,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",2630429,2630555,126,0.331975,null,null,null,108.837795,204.895276,5,"""t""",95.0,184.7,14,33,1,1,1,0,"""https://sq.wikipedia.org/wiki/…",true,1,1
1,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",2630834,2631374,540,1.651275,null,null,null,121.55915,214.301479,6,"""t""",109.0,184.7,14,33,2,1,1,0,"""https://sq.wikipedia.org/wiki/…",false,1,1
2,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",2634207,2634315,108,0.683358,null,null,null,186.726606,312.337615,45,"""i""",179.0,280.4,14,33,7,2,3,1,"""Wikipedia,""",true,2,1
3,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",2635216,2635416,200,0.50865,null,null,null,349.59801,311.577612,57,"""k""",347.0,280.4,14,33,19,2,4,2,"""enciklopedia""",true,3,1
4,"""PRACTICE_trial_1""","""Enc_WikiMoon_13""","""page_1""","""fixation""",2636548,2636854,306,1.298191,null,null,null,109.493485,397.897068,74,"""n""",109.0,376.1,14,33,2,3,7,0,"""Hëna""",true,4,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4518,"""trial_9""","""PopSci_MultiplEYE_1""","""page_9""","""fixation""",9507281,9507427,146,0.524145,null,null,null,311.618367,855.941497,639,"""a""",305.0,854.6,14,33,16,8,101,2,"""përmirësuar""",true,33,1
4519,"""trial_9""","""PopSci_MultiplEYE_1""","""page_9""","""fixation""",9507470,9507600,130,0.578337,null,null,null,406.038168,869.743511,646,"""r""",403.0,854.6,14,33,23,8,102,3,"""nxjerrjen""",true,34,1
4520,"""trial_9""","""PopSci_MultiplEYE_1""","""page_9""","""fixation""",9507956,9508299,343,1.464704,null,null,null,914.547384,876.592151,682,""" """,907.0,854.6,14,33,59,8,107,8,""" """,true,35,1


In [84]:
trial = "trial_4"
page = "page_1"

fix_tp = fixation_table.filter(
    (pl.col('trial') == trial) & (pl.col('page') == page)
)

In [85]:
all_words = all_words_from_aois(aois, page)

words_with_skip = find_skipped_words(all_words, fix_tp)

In [86]:
word_level_table = build_word_level_table(
    words=words_with_skip.with_columns([
        pl.lit(trial).alias("trial"),
        pl.lit(page).alias("page"),
    ]),
    fix=fix_tp,
)

In [87]:
word_level_table

page,word_idx,word,skipped,trial,TFC,FFD,FPRT,RRT,FPFC,TFT,FPF,RR,SFD
str,i64,str,i8,str,u32,i64,i64,i64,u32,i64,i8,i8,i64
"""page_1""",0,"""Mali""",0,"""trial_4""",3,115,115,521,1,636,1,1,115
"""page_1""",0,""" """,0,"""trial_4""",3,115,115,521,1,636,1,1,115
"""page_1""",1,"""Magjik""",0,"""trial_4""",2,327,327,230,1,557,1,1,327
"""page_1""",1,""" """,0,"""trial_4""",2,327,327,230,1,557,1,1,327
"""page_1""",2,"""-""",1,"""trial_4""",0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""page_1""",106,""" """,0,"""trial_4""",2,174,295,0,2,295,1,0,0
"""page_1""",106,"""tejshkuar""",0,"""trial_4""",2,174,295,0,2,295,1,0,0
"""page_1""",107,""" """,1,"""trial_4""",0,0,0,0,0,0,0,0,0


### Transition-based Metrics

## The END


	-- data collection folder
	---- ...
	---- fixations
	---- saccades(?)
	---- reading_measures
	---- raw_data (i.e. gaze sample csv)